<a href="https://colab.research.google.com/github/stabti/computer_vision_sessions/blob/main/notebooks/Visualize_and_Cluster_Embeddings_with_FiftyOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial - Visualize Embedding Spaces with FiftyOne

#### Author: Antonio Rueda-Toicen
**antonio.rueda.toicen 'at' hpi 'dot' de**


[![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)](http://creativecommons.org/licenses/by/4.0/)

This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

## Overview

In this notebook we learn how to visualize an embedding space produced with a [ResNet34](https://docs.pytorch.org/vision/stable/models/generated/torchvision.models.resnet34.html) using [FiftyOne](https://docs.voxel51.com/) after applying [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) to reduce its dimensionality. We also use `sklearn` and `sklearn-extra` to produce clusters using the [K-means](https://en.wikipedia.org/wiki/K-means_clustering) algorithm.

![]()


## Image data
The folder with the image data and can be found [here](https://drive.google.com/drive/folders/1oZOMfxEYcrYctZSdx3NHO8KTc0ETrUFI?usp=drive_link).

You can add it to your own Google Drive by **right clicking on the folder name** -> **Organize** -> **Add Shortcut to Drive**. Select the **"All locations"** tab -> **My Drive** and then create a folder called `art_recommendation`. This will allow you to access the data without having to download it. More details [here](https://github.com/andandandand/practical-computer-vision?tab=readme-ov-file).

In [1]:
# Install fiftyone
!pip install fiftyone==1.7.0 > /dev/null


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.21.0 requires sse-starlette>=1.6.1, but you have sse-starlette 0.10.3 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
from pathlib import Path
import os
artist_name = 'Hokusai'
path = Path(f'/gdrive/MyDrive/art_recommendation/{artist_name}')

In [5]:
# here 'paintings' and 'paintings_embeddings.pickle' should both appear
os.listdir(path)

['paintings', 'Hokusai artworks', 'paintings_embeddings.pickle', 'images.zip']

In [6]:
# Path where the images are
images_dir = path / "paintings"
images_dir

PosixPath('/gdrive/MyDrive/art_recommendation/Hokusai/paintings')

In [7]:
# Number of images
len(os.listdir(images_dir))

832

## Create a fiftyone dataset

If fiftyone, datasets are a collection of images to which we can add metadata (labels, quality metrics, embeddings, etc.).

In [8]:
import fiftyone as fo


dataset_name = dataset_name = f"{artist_name}_paintings"

# delete the dataset in case it exists already on the Colab instance
# (due to multiple evaluations of the code cell)
if fo.dataset_exists(dataset_name):
  fo.delete_dataset(dataset_name)

/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'


In [9]:
# this creates an empty dataset
dataset = fo.Dataset(dataset_name)
dataset

Name:        Hokusai_paintings
Media type:  None
Num samples: 0
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField

In [10]:
# We use the location of the images_dir to add samples to the dataset
dataset.add_dir(images_dir, dataset_type=fo.types.ImageDirectory)

 100% |█████████████████| 832/832 [315.7ms elapsed, 0s remaining, 2.7K samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 832/832 [315.7ms elapsed, 0s remaining, 2.7K samples/s]      


['6918be369cf4642f04d3fca9',
 '6918be369cf4642f04d3fcaa',
 '6918be369cf4642f04d3fcab',
 '6918be369cf4642f04d3fcac',
 '6918be369cf4642f04d3fcad',
 '6918be369cf4642f04d3fcae',
 '6918be369cf4642f04d3fcaf',
 '6918be369cf4642f04d3fcb0',
 '6918be369cf4642f04d3fcb1',
 '6918be369cf4642f04d3fcb2',
 '6918be369cf4642f04d3fcb3',
 '6918be369cf4642f04d3fcb4',
 '6918be369cf4642f04d3fcb5',
 '6918be369cf4642f04d3fcb6',
 '6918be369cf4642f04d3fcb7',
 '6918be369cf4642f04d3fcb8',
 '6918be369cf4642f04d3fcb9',
 '6918be369cf4642f04d3fcba',
 '6918be369cf4642f04d3fcbb',
 '6918be369cf4642f04d3fcbc',
 '6918be369cf4642f04d3fcbd',
 '6918be369cf4642f04d3fcbe',
 '6918be369cf4642f04d3fcbf',
 '6918be369cf4642f04d3fcc0',
 '6918be369cf4642f04d3fcc1',
 '6918be369cf4642f04d3fcc2',
 '6918be369cf4642f04d3fcc3',
 '6918be369cf4642f04d3fcc4',
 '6918be369cf4642f04d3fcc5',
 '6918be369cf4642f04d3fcc6',
 '6918be369cf4642f04d3fcc7',
 '6918be369cf4642f04d3fcc8',
 '6918be369cf4642f04d3fcc9',
 '6918be369cf4642f04d3fcca',
 '6918be369cf4

In [11]:
# add metadata on file size, image format, and image dimensions
dataset.compute_metadata()


Computing metadata...


INFO:fiftyone.core.metadata:Computing metadata...


 100% |█████████████████| 832/832 [34.4s elapsed, 0s remaining, 304.8 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 832/832 [34.4s elapsed, 0s remaining, 304.8 samples/s]      


## Connect embeddings to the dataset

In [12]:
import pickle
with open(path /'paintings_embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)


assert len(embeddings) == len(os.listdir(images_dir))


/tmp/ipython-input-2728714600.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  embeddings = pickle.load(f)


In [14]:
for sample in dataset:
  # Get the filename from sample.filepath
  filename = Path(sample.filepath).name
  # Construct the correct key for the embeddings dictionary
  embedding_key = images_dir / filename
  sample["embedding"] = embeddings[embedding_key]
  sample.save()

dataset.save()

In [15]:
# We can see that the embedding field has been added
dataset.view()

Dataset:     Hokusai_paintings
Media type:  image
Num samples: 832
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    embedding:        fiftyone.core.fields.VectorField
View stages:
    ---

## Principal Component Analysis

In [23]:
import fiftyone.brain as fob

embeddings = dataset.values('embedding')

pca_brain_key = f"{dataset_name}_pca"

# Check if brain key exists and delete it to recompute with labels
if dataset.has_brain_run(pca_brain_key):
    dataset.delete_brain_run(pca_brain_key)
    dataset.save()

# Compute 2D representation, now including clustering labels
results = fob.compute_visualization(
    dataset,
    embeddings=embeddings,
    num_dims=2,
    method="pca",
    brain_key=pca_brain_key,
    labels_field="k_means_cluster",  # Add this to visualize clusters
    verbose=True,
    seed=51,
)

Generating visualization...


INFO:fiftyone.brain.visualization:Generating visualization...


## Clustering

In [24]:
import numpy as np
from sklearn.cluster import KMeans
import fiftyone as fo


# Choose the number of clusters
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

# Assign each sample its cluster label
for sample, label in zip(dataset, labels):
    sample["k_means_cluster"] = int(label)
    sample.save()

print("Clusters computed and stored in each sample's 'cluster' field.")


Clusters computed and stored in each sample's 'cluster' field.


In [25]:
# Save the dataset to keep views in sync
dataset.save()

## Launch the FiftyOne App

In [26]:
# Launch the FiftyOne App, if you get an error try going into Runtime -> Run All
# If that fails, go to Runtime -> Disconnect and Delete Runtime
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [27]:
# Copy and paste the session url in a separate browser window
print(session.url)

https://5151-m-s-27ow0i9mepxbp-a.us-east4-0.prod.colab.dev?polling=true


![]()